In [1]:
### 라이브러리 불러오기 

In [2]:
!pip install tqdm

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.0.1 -> 23.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
from PIL import Image
import seaborn as sns
import time
import copy
from glob import glob
import cv2
import shutil
import os
from tqdm.notebook import tqdm
import numpy as np





In [2]:
from os.path import join

In [4]:
!pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.0.1 -> 23.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import normalize
from sklearn.decomposition import PCA

데이터셋 중 하나의 데이터 테스트로 하나 가져와보기 

In [6]:
# 훈련데이터셋 경로 설정 
train_dataset_dir = 'data/pca_data1/test/'
# 테스트 데이터셋 경로 설정 
test_dataset_dir = 'data/pca_data1/train/'
# 훈련 데이터 추출
train_data = sorted(glob(join(train_dataset_dir, '*/*.png')))
print(train_data)
# 테스트 데이터 추출
test_data = sorted(glob(join(test_dataset_dir, '*/*.png')))
print(test_data)


['data/pca_data1/test\\Ariel_Sharon\\00024.png', 'data/pca_data1/test\\Ariel_Sharon\\00052.png', 'data/pca_data1/test\\Ariel_Sharon\\00081.png', 'data/pca_data1/test\\Ariel_Sharon\\00123.png', 'data/pca_data1/test\\Ariel_Sharon\\00157.png', 'data/pca_data1/test\\Ariel_Sharon\\00168.png', 'data/pca_data1/test\\Ariel_Sharon\\00209.png', 'data/pca_data1/test\\Ariel_Sharon\\00217.png', 'data/pca_data1/test\\Ariel_Sharon\\00218.png', 'data/pca_data1/test\\Ariel_Sharon\\00233.png', 'data/pca_data1/test\\Ariel_Sharon\\00245.png', 'data/pca_data1/test\\Ariel_Sharon\\00285.png', 'data/pca_data1/test\\Ariel_Sharon\\00295.png', 'data/pca_data1/test\\Colin_Powell\\00007.png', 'data/pca_data1/test\\Colin_Powell\\00020.png', 'data/pca_data1/test\\Colin_Powell\\00025.png', 'data/pca_data1/test\\Colin_Powell\\00039.png', 'data/pca_data1/test\\Colin_Powell\\00041.png', 'data/pca_data1/test\\Colin_Powell\\00043.png', 'data/pca_data1/test\\Colin_Powell\\00044.png', 'data/pca_data1/test\\Colin_Powell\\000

In [7]:
print(len(train_data), len(test_data))
# 훈련 데이터 개수 : 322개 test data개수 : 966개


322 966


In [10]:
# matplolib 설정해주기 
plt.rcParams['axes.unicode_minus']
matplotlib.rc('font', family = 'AppleGothic')

In [16]:
# train 이미지 데이터 전체를 로딩하기 
list_data = []
for i in tqdm(train_data):
    list_data.append(cv2.imread(i ,1))
np_imgs = np.array(list_data)
print(np_imgs.shape)

  0%|          | 0/322 [00:00<?, ?it/s]

(322, 50, 37, 3)


In [19]:
def show(img_bgr) :
    plt.imshow(cv2.cvtColor(img_bgr, cv2.COLOR_BAYER_BG2RGB))
    plt.show()

In [20]:
show(list_data[0])

error: OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\demosaicing.cpp:1721: error: (-215:Assertion failed) scn == 1 && (dcn == 3 || dcn == 4) in function 'cv::demosaicing'


# 이미지 데이터 전처리 준비하기 

In [13]:
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim 
from torch.utils.data import dataloader
import matplotlib.pyplot as plt

In [21]:
import time
import copy
import glob
import cv2
import shutil

이미지 데이터 전처리 방식 정의하기 

In [23]:
transform = transforms.Compose(
    [
        transforms.Resize([256,256]),
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor()
    ]
)

In [26]:
train_dataset = torchvision.datasets.ImageFolder(
    train_dataset_dir, 
    transform = transform
)

In [28]:
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size = 128,
    num_workers = 8,
    shuffle = True,
)

print(len(train_dataset))

322


C:\Users\trixy\AppData\Roaming\Python\Python310\site-packages\torch\utils\data\dataloader.py:561: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4 (`cpuset` is not taken into account), which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


데이터 행렬 형태로 바꿔주기

In [30]:
labels, _ = next(iter(train_loader))
classes = {0:'Ariel_Sharon', 1:'Colin_Powell', 2:'Donald_Rumsfeld', 2:'George_W_Bush', 3 : 'Gerhard_Shcroeder', 4:'Hugo_Chavez', 5:'Tony_Blair'}
fig = plt.figure(figsize = (16,24))


C:\Users\trixy\AppData\Roaming\Python\Python310\site-packages\torch\utils\data\dataloader.py:561: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4 (`cpuset` is not taken into account), which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
